<a href="https://colab.research.google.com/github/Ryoo1014/Ryoo1014/blob/main/%E5%82%B5%E5%8B%99%E4%B8%8D%E5%B1%A5%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [3]:
def write_submission(id, pred, output_file):
    """予測結果を出力します."""
    df = pd.DataFrame({'id': id, 'price_range': pred})
    df.to_csv(output_file, index=False, header=False)

In [4]:
train_df = pd.read_csv('/content/債務不履行/train.csv')
test_df = pd.read_csv('/content/債務不履行/test.csv')

In [5]:
train_df

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,88194295,1800.0,3 years,14.49,C4,NaN,debt_consolidation,665.0,Individual,FullyPaid
1,5146039,1200.0,5 years,16.29,C4,2 years,debt_consolidation,700.0,Individual,ChargedOff
2,3095896,2000.0,5 years,21.98,E4,10 years,home_improvement,670.0,Individual,FullyPaid
3,88625044,1000.0,3 years,8.59,A5,4 years,debt_consolidation,710.0,Individual,FullyPaid
4,1178189,1500.0,3 years,13.99,C1,4 years,debt_consolidation,680.0,Individual,FullyPaid
...,...,...,...,...,...,...,...,...,...,...
113683,1351731,1600.0,3 years,12.12,B3,10 years,debt_consolidation,700.0,Individual,FullyPaid
113684,91545359,800.0,3 years,13.99,C3,10 years,debt_consolidation,680.0,Individual,FullyPaid
113685,94362123,2990.0,5 years,26.24,E5,10 years,home_improvement,670.0,Individual,ChargedOff
113686,91297171,600.0,3 years,19.99,D4,10 years,debt_consolidation,675.0,Individual,ChargedOff


from matplotlib import pyplot as plt
train_df['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
train_df['loan_amnt'].plot(kind='hist', bins=20, title='loan_amnt')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
train_df['interest_rate'].plot(kind='hist', bins=20, title='interest_rate')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
train_df['credit_score'].plot(kind='hist', bins=20, title='credit_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['loan_status'].value_counts()
    for x_label, grp in train_df.groupby('application_type')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('application_type')
_ = plt.ylabel('loan_status')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(train_df['term'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(train_df, x='id', y='term', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(train_df['application_type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(train_df, x='id', y='application_type', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(train_df['loan_status'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(train_df, x='id', y='loan_status', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(train_df['term'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(train_df, x='loan_amnt', y='term', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)